In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model,layers
import tensorflow as tf
import keras.backend as K
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
#from keras.utils import np_utils
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, SpatialDropout1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
#import os os.environ['CUDA_VISIBLE_DEVICES']=1
from keras import callbacks
from keras.callbacks import CSVLogger

#from utilss import utils

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'
'''
config = tf.compat.v1.ConfigProto()
config.gpu_compat.v1.options.allow_growth = True
tf.compat.v1.keras.backend.tensorflow_backend.set_session(tf.Session(config=config))
'''

gpu_options=tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.85)
config=tf.compat.v1.ConfigProto(gpu_options=gpu_options)
session = tf.compat.v1.Session(config=config)




T = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\split3new_s3b.csv", header=None)
C = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\split3news3_benign.csv",header=None)
f3 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\benign\benign3_b.csv", header=None)
T = T.append (f3)

T1 = T
T1.fillna(0,inplace=True)
T1.replace([np.inf,-np.inf],1,inplace=True)
testT = np.array(T1)
y_test = np.array(C)
X_test = np.reshape(testT, (testT.shape[0],32,24))
# Add a channels dimension
#X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")
new_model = tf.keras.models.load_model('saved_model/my_modelb')

# Check its architecture
new_model.summary()
# Evaluate the restored model
optimizer = tf.keras.optimizers.Adam()

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

new_model.compile(optimizer, test_loss)
loss, acc = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(X_test).shape)


class myMetrics:
    def __init__(self, X_test, Y_test, model):
        self.model = model
        self.Y_test = Y_test
        self.X_test = X_test
        self.y_pred = self.model.predict(self.X_test)

    def Precision(self):
        """精确率"""
        self.Y_test=self.Y_test.astype ( np.float32)
        tp = K.sum(K.round(K.clip(self.Y_test * self.y_pred, 0, 1)))  # true positives
        pp = K.sum(K.round(K.clip(self.y_pred, 0, 1)))  # predicted positives
        '''
        print ( type (tp))
        print ( tp.dtype)
        print ( type (pp))
        print ( pp.dtype)
        print ( type (K.epsilon()))
        '''
        
        #print ( K.epsilon().dtype)
        precision = tp / (pp + K.epsilon())
        self.precision = precision
        return precision

    def Recall(self):
        """召回率"""
        tp = K.sum(K.round(K.clip(self.Y_test * self.y_pred, 0, 1)))  # true positives
        pp = K.sum(K.round(K.clip(self.Y_test, 0, 1)))  # possible positives
        recall = tp / (pp + K.epsilon())
        self.recall = recall
        return recall

    def F1(self):
        """F1-score"""
        try:
            precision = self.precision
            recall = self.recall
        except:
            precision = self.Precision()
            recall = self.Recall()
        f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
        self.f1 = f1
        return f1

    def LossAccuracy(self):
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=Adam(lr=1e-4),
                           metrics=['accuracy'])
        metric = self.model.evaluate(x=self.X_test, y=self.Y_test, verbose=0)
        print(self.model.metrics_names)
        self.accuracy = metric[1]
        self.loss = metric[0]
        return metric

    def auto(self):
        self.Precision()
        self.Recall()
        self.F1()
        self.LossAccuracy()
        return self.precision, self.recall, self.f1, self.accuracy, self.loss


def mktest(lines=None, size=(224, 224)):
    if lines == None:
        with open('../../../Metrics/cate.txt', 'r') as l:
            lines = l.readlines()
            l.close()
    img_dir = '../../../Metrics/imgs/'
    imgs = []
    gt = []
    for line in lines:
        imgs.append(img_dir + line.split(';')[0])
        gt.append(int(line.split(';')[1]))
    X_test = []
    Y_test = gt
    for il in imgs:
        img = cv2.imread(il)
        try:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            pass

        img = np.float32(img / 255)
        X_test.append(img)
    X_test = utils.resize_image(X_test, (224, 224))
    X_test = X_test.reshape(-1, 224, 224, 3)
    Y_test = np_utils.to_categorical(np.array(Y_test), num_classes=6)
    return X_test, Y_test


def get_pre(Y_test, X_test):
    Y_pre = model.predict(X_test)
    np.savez('./data.npz', X_test, Y_test, Y_pre)
    return Y_pre


if __name__ == '__main__':
    #data, gt = mktest()
    data, gt = X_test,y_test
    # get_pre(gt, data)
    mym = myMetrics(data, gt, new_model)
    precision, recall, f1, loss, accuracy = mym.auto()
    print ( precision)
    print ( recall )
    print ( f1)
    print ( loss)
    print ( accuracy )
    #np.savez('./result.npz', precision, recall, f1, loss, accuracy)


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  32832     
                                                                 
 conv2d (Conv2D)             multiple                  320       
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           multiple                  18496     
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                      

C:\d\anaconda\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


['loss', 'accuracy', 'test_loss']
tf.Tensor(0.90330976, shape=(), dtype=float32)
tf.Tensor(0.88303196, shape=(), dtype=float32)
tf.Tensor(0.89305574, shape=(), dtype=float32)
0.89439857006073
0.22012503445148468


In [5]:
T1 = pd.read_csv(r"C:\Users\maijieai\pratice\p1\tensorf\gan_try\samplest\4.csv")
C = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\sample2.csv", header=None)
print ( C.shape)
#C = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\sample2.csv", header=None)
T1.fillna(0,inplace=True)
T1.replace([np.inf,-np.inf],1,inplace=True)
T1 = T1.iloc[:,1:769]
testT = np.array(T1)
#y_test = np.array(C)
X_test = np.reshape(testT, (testT.shape[0],32,24))
y_test = np.array(C)

# Add a channels dimension
#X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")
if __name__ == '__main__':
    #data, gt = mktest()
    data, gt = X_test,y_test
    # get_pre(gt, data)
    mym = myMetrics(data, gt, new_model)
    precision, recall, f1, loss, accuracy = mym.auto()
    print ( precision.numpy())
    print ( recall.numpy() )
    print ( f1.numpy())
    print ( loss)
    print ( accuracy )

(14461, 10)
['loss', 'accuracy', 'test_loss']
0.050065693
0.050065693
0.05006564
0.05006569251418114
15.31117057800293


In [6]:
precision, recall, f1, loss, accuracy = mym.auto()
print ( precision)
print ( recall )
print ( f1)
print ( loss)
print ( accuracy )

['loss', 'accuracy', 'test_loss']
tf.Tensor(0.050065693, shape=(), dtype=float32)
tf.Tensor(0.050065693, shape=(), dtype=float32)
tf.Tensor(0.05006564, shape=(), dtype=float32)
0.05006569251418114
15.31117057800293
